In [111]:
import os # for making directories
import requests

# make a directory for pdb files
os.makedirs("pdb", exist_ok=True)

pdb_id = "2zq2" # trypsin PDB file with ligand bound

pdb_request = requests.get(f"https://files.rcsb.org/download/{pdb_id}.pdb")
pdb_request.status_code

200

In [113]:
with open(f"pdb/{pdb_id}.pdb", "w+") as f:
    f.write(pdb_request.text)

In [115]:
import math

def rotate_view(view, x=0, y=0, z=0, degrees=True):
    radians = 1
    if degrees: radians = math.pi / 180
    view.control.spin([1, 0, 0], x*radians)
    view.control.spin([0, 1, 0], y*radians)
    view.control.spin([0, 0, 1], z*radians)

def view_binding_site(protein, ligand):
    """View binding site of 13U to trypsin.

    Parameters
    -----------
    protein: mda.Universe
        The protein as an MDAnalysis universe.

    ligand: mda.Universe
        The ligand as an MDAnalysis universe.
    """
    view = nv.show_mdanalysis(protein)
    view.clear_representations()
    view.add_representation("surface", colorScheme="hydrophobicity")
    lig_view = view.add_component(ligand)
    lig_view.center()
    rotate_view(view, y=180, x=20)
    return view
    

In [117]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe(f"pdb/{pdb_id}.pdb")

In [119]:
view = nv.show_mdanalysis(u)
view

NGLWidget()

In [121]:
protein = u.select_atoms("protein")
ligand = u.select_atoms("resname 13U")

In [123]:
view_binding_site(protein, ligand)

NGLWidget()

In [125]:
protein = u.select_atoms("protein")
ligand_A = u.select_atoms("resname 13U and altLoc A")
ligand_B = u.select_atoms("resname 13U and altLoc B")

In [127]:
view_binding_site(protein, ligand_A)

NGLWidget()

In [129]:
view_binding_site(protein, ligand_B)

NGLWidget()

In [131]:
protein.write(f"pdb/protein_{pdb_id}.pdb")
ligand_A.write(f"pdb/ligand_A.pdb")

C:\Users\Eric\anaconda3\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [133]:
! pdb2pqr --pdb-output=pdb/protein_h.pdb --pH=7.4 pdb/protein_2zq2.pdb protein.pqr

INFO:PDB2PQR v3.7.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: pdb/protein_2zq2.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 223 residues and 1625 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 4 missing atoms in biomolecule.
INFO:Added atom CG to residue LYS A 222 at coordinates 30

In [135]:
protein = mda.Universe("pdb/protein_h.pdb")

In [149]:
from rdkit import Chem

from rdkit.Chem.AllChem import AssignBondOrdersFromTemplate

template = Chem.MolFromMol2File("ligands/13U_ideal.mol2")
pdb_ligand = Chem.MolFromPDBFile(f"pdb/ligand_A.pdb")

template = Chem.RemoveAllHs(pdb_ligand)

In [157]:
ligand = AssignBondOrdersFromTemplate(template, pdb_ligand)

# Write the ligand to an SDF file
Chem.MolToMolFile(ligand, "pdb/ligand.sdf")

In [166]:
from openbabel import pybel

# Use pybel to read the SDF, add hydrogens, and save as PDB
mol = next(pybel.readfile("sdf", "pdb/ligand.sdf"))
mol.addh()  # Add hydrogens
mol.write("pdb", "pdb/ligand_h.pdb", overwrite=True)

ModuleNotFoundError: No module named 'openbabel'

In [171]:
import prolif as plf

protein_h = mda.Universe(f"pdb/protein_h.pdb")
ligand_h = mda.Universe(f"pdb/ligand_h.pdb")

FileNotFoundError: [Errno 2] No such file or directory: 'pdb/ligand_h.pdb'

In [175]:
protein_mol = plf.Molecule.from_mda(protein_h)

NameError: name 'RDATTRIBUTES' is not defined

In [177]:
ligand_mol = plf.Molecule.from_mda(ligand_h)

NameError: name 'ligand_h' is not defined

In [180]:
fp = plf.Fingerprint()

In [182]:
lig_list = [ ligand_mol ] 

interactions = fp.run_from_iterable(lig_list, protein_mol)

NameError: name 'ligand_mol' is not defined

In [184]:
view = fp.plot_lignetwork(lig_list[0])
view

NameError: name 'lig_list' is not defined